<a href="https://colab.research.google.com/github/xbeafraid1/ILTAL/blob/main/Python_GBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#지금 만들어야 하는 모델 : 
CatBoost, AdaBoost, XGBoost, LGBM 중 어느 것을 쓸 지 아직 정해지지 않았지만 공통적으로 적용될 수 있는 모델을 만들어보려고 함.



### 5일, 4일, 3일, 2일, 1일 중 어느 것으로 앞의 것들을 예측해볼 지는 아직 정해지지 않음.


In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import time
import torch
import statsmodels.api as sm
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.regression.quantile_regression import QuantReg
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from pandas import DataFrame, concat
import torch
import os
import import_ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# !cp를 이용하여 기존에 있었던 함수를 불러온다.
# GB_functions.ipynb에 전처리 함수를 저장해 놓았으므로 얘를 JY로 불러온다.
!cp /content/drive/MyDrive/Jupyter/unlimited_power/working__/GB_functions.ipynb .

import GB_functions as JY

importing Jupyter notebook from GB_functions.ipynb


In [20]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 3                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_use = ['Time', 'DHI','DNI','RH','T','TARGET']

n_features = len(what_to_use)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [47]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')

In [12]:
# drop_clms 함수를 이용하여 쓰지 않는 데이터 분리하기

dataset = JY.drop_clms(raw_data)

In [13]:
dataset

,Time,DHI,DNI,RH,T,TARGET
0,0.0,0,0,69.08,-12,0.0
1,0.5,0,0,69.06,-12,0.0
2,1.0,0,0,71.78,-12,0.0
3,1.5,0,0,71.75,-12,0.0
4,2.0,0,0,75.20,-12,0.0
...,...,...,...,...,...,...
52555,21.5,0,0,70.70,-4,0.0
52556,22.0,0,0,66.79,-4,0.0
52557,22.5,0,0,66.78,-4,0.0
52558,23.0,0,0,67.72,-4,0.0


In [36]:
# dataset, n_days 2개의 변수를 input으로 받음,  앞의 변수는 raw_data /  뒤의 변수는 이전 며칠을 불러올 것인지 정의한 것.

dataset2 = JY.series_to_supervised(dataset, n_days)

In [72]:
dataset2

,var1(t-144),var2(t-144),var3(t-144),var4(t-144),var5(t-144),var6(t-144),var1(t-143),var2(t-143),var3(t-143),var4(t-143),var5(t-143),var6(t-143),var1(t-142),var2(t-142),var3(t-142),var4(t-142),var5(t-142),var6(t-142),var1(t-141),var2(t-141),var3(t-141),var4(t-141),var5(t-141),var6(t-141),var1(t-140),var2(t-140),var3(t-140),var4(t-140),var5(t-140),var6(t-140),var1(t-139),var2(t-139),var3(t-139),var4(t-139),var5(t-139),var6(t-139),var1(t-138),var2(t-138),var3(t-138),var4(t-138),...,TARGET1(t+56),TARGET1(t+57),TARGET1(t+58),TARGET1(t+59),TARGET1(t+60),TARGET1(t+61),TARGET1(t+62),TARGET1(t+63),TARGET1(t+64),TARGET1(t+65),TARGET1(t+66),TARGET1(t+67),TARGET1(t+68),TARGET1(t+69),TARGET1(t+70),TARGET1(t+71),TARGET1(t+72),TARGET1(t+73),TARGET1(t+74),TARGET1(t+75),TARGET1(t+76),TARGET1(t+77),TARGET1(t+78),TARGET1(t+79),TARGET1(t+80),TARGET1(t+81),TARGET1(t+82),TARGET1(t+83),TARGET1(t+84),TARGET1(t+85),TARGET1(t+86),TARGET1(t+87),TARGET1(t+88),TARGET1(t+89),TARGET1(t+90),TARGET1(t+91),TARGET1(t+92),TARGET1(t+93),TARGET1(t+94),TARGET1(t+95)
144,0.0,0.0,0.0,69.08,-12.0,0.0,0.5,0.0,0.0,69.06,-12.0,0.0,1.0,0.0,0.0,71.78,-12.0,0.0,1.5,0.0,0.0,71.75,-12.0,0.0,2.0,0.0,0.0,75.20,-12.0,0.0,2.5,0.0,0.0,69.29,-11.0,0.0,3.0,0.0,0.0,72.56,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,4.504698,9.478531,4.129216,7.789117,9.102844,12.105844,25.056006,17.079375,25.806465,20.645172,18.111446,25.149849,15.671734,16.422476,9.196587,6.381375,3.566063,1.032293,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
145,0.5,0.0,0.0,69.06,-12.0,0.0,1.0,0.0,0.0,71.78,-12.0,0.0,1.5,0.0,0.0,71.75,-12.0,0.0,2.0,0.0,0.0,75.20,-12.0,0.0,2.5,0.0,0.0,69.29,-11.0,0.0,3.0,0.0,0.0,72.56,-11.0,0.0,3.5,0.0,0.0,72.55,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,4.504698,9.478531,4.129216,7.789117,9.102844,12.105844,25.056006,17.079375,25.806465,20.645172,18.111446,25.149849,15.671734,16.422476,9.196587,6.381375,3.566063,1.032293,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,1.0,0.0,0.0,71.78,-12.0,0.0,1.5,0.0,0.0,71.75,-12.0,0.0,2.0,0.0,0.0,75.20,-12.0,0.0,2.5,0.0,0.0,69.29,-11.0,0.0,3.0,0.0,0.0,72.56,-11.0,0.0,3.5,0.0,0.0,72.55,-11.0,0.0,4.0,0.0,0.0,74.62,...,0.0,0.0,0.0,0.0,0.000000,0.000000,4.504698,9.478531,4.129216,7.789117,9.102844,12.105844,25.056006,17.079375,25.806465,20.645172,18.111446,25.149849,15.671734,16.422476,9.196587,6.381375,3.566063,1.032293,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,1.5,0.0,0.0,71.75,-12.0,0.0,2.0,0.0,0.0,75.20,-12.0,0.0,2.5,0.0,0.0,69.29,-11.0,0.0,3.0,0.0,0.0,72.56,-11.0,0.0,3.5,0.0,0.0,72.55,-11.0,0.0,4.0,0.0,0.0,74.62,-11.0,0.0,4.5,0.0,0.0,74.61,...,0.0,0.0,0.0,0.0,0.000000,4.504698,9.478531,4.129216,7.789117,9.102844,12.105844,25.056006,17.079375,25.806465,20.645172,18.111446,25.149849,15.671734,16.422476,9.196587,6.381375,3.566063,1.032293,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,2.0,0.0,0.0,75.20,-12.0,0.0,2.5,0.0,0.0,69.29,-11.0,0.0,3.0,0.0,0.0,72.56,-11.0,0.0,3.5,0.0,0.0,72.55,-11.0,0.0,4.0,0.0,0.0,74.62,-11.0,0.0,4.5,0.0,0.0,74.61,-11.0,0.0,5.0,0.0,0.0,73.74,...,0.0,0.0,0.0,0.0,4.504698,9.478531,4.129216,7.789117,9.102844,12.105844,25.056006,17.079375,25.806465,20.645172,18.111446,25.149849,15.671734,16.422476,9.196587,6.381375,3.566063,1.032293,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52460,22.0,0.0,0.0,82.26,-8.0,0.0,22.5,0.0,0.0,82.25,-8.0,0.0,23.0,0.0,0.0,81.14,-8.0,0.0,23.5,0.0,0.0,81.12,-8.0,0.0,0.0,0.0,0.0,80.36,-8.0,0.0,0.5,0.0,0.0,74.27

In [74]:
future_window

96

In [85]:
# train_X, train_y, test_X, test_y로 분리하는 함수.

X = dataset2.iloc[:, :n_obs]
Y1 = dataset2.iloc[:, -49] #  2일동안 마지막 틱만 추출하여 예측값으로 설정한다. 
Y2 = dataset2.iloc[:, -1]

# 7 : 3으로 Train Set :  Test Set을, 7 : 3으로 Trai

X_train_1, X_test_1, Y_train_1, Y_test_1 = train_test_split(X, Y1, test_size=0.30, random_state=42)
X_train_2, X_test_2, Y_train_2, Y_test_2 = train_test_split(X, Y2, test_size=0.30, random_state=42)

sklearn.model_selection.train_test_split두 번만 사용할 수 있습니다 . 먼저 기차로 나누고 테스트 한 다음 기차를 다시 유효성 검사와 기차로 나눕니다. 이 같은:

 X_train, X_test, y_train, y_test 
    = train_test_split(X, y, test_size=0.2, random_state=1)

 X_train, X_val, y_train, y_val 
    = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

,var1(t-144),var2(t-144),var3(t-144),var4(t-144),var5(t-144),var6(t-144),var1(t-143),var2(t-143),var3(t-143),var4(t-143),var5(t-143),var6(t-143),var1(t-142),var2(t-142),var3(t-142),var4(t-142),var5(t-142),var6(t-142),var1(t-141),var2(t-141),var3(t-141),var4(t-141),var5(t-141),var6(t-141),var1(t-140),var2(t-140),var3(t-140),var4(t-140),var5(t-140),var6(t-140),var1(t-139),var2(t-139),var3(t-139),var4(t-139),var5(t-139),var6(t-139),var1(t-138),var2(t-138),var3(t-138),var4(t-138),...,var3(t-7),var4(t-7),var5(t-7),var6(t-7),var1(t-6),var2(t-6),var3(t-6),var4(t-6),var5(t-6),var6(t-6),var1(t-5),var2(t-5),var3(t-5),var4(t-5),var5(t-5),var6(t-5),var1(t-4),var2(t-4),var3(t-4),var4(t-4),var5(t-4),var6(t-4),var1(t-3),var2(t-3),var3(t-3),var4(t-3),var5(t-3),var6(t-3),var1(t-2),var2(t-2),var3(t-2),var4(t-2),var5(t-2),var6(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1)
38122,5.0,0.0,0.0,56.51,-3.0,0.000000,5.5,0.0,0.0,56.50,-3.0,0.000000,6.0,0.0,0.0,55.54,-3.0,0.000000,6.5,0.0,0.0,47.81,-1.0,0.000000,7.0,28.0,388.0,47.75,0.0,5.630873,7.5,43.0,627.0,44.37,1.0,14.640108,8.0,51.0,756.0,43.35,...,0.0,63.19,3.0,0.000000,2.0,0.0,0.0,60.08,3.0,0.000000,2.5,0.0,0.0,64.52,2.0,0.000000,3.0,0.0,0.0,60.50,2.0,0.000000,3.5,0.0,0.0,60.48,1.0,0.000000,4.0,0.0,0.0,58.56,1.0,0.000000,4.5,0.0,0.0,58.59,1.0,0.000000
4783,15.5,133.0,712.0,18.15,16.0,49.167641,16.0,134.0,583.0,18.52,16.0,38.189370,16.5,55.0,809.0,19.76,14.0,33.873871,17.0,50.0,701.0,30.45,13.0,23.458617,17.5,67.0,220.0,37.18,10.0,10.134457,18.0,33.0,253.0,45.58,8.0,5.161179,18.5,0.0,0.0,52.34,...,945.0,19.79,15.0,88.108630,12.5,133.0,942.0,19.78,15.0,87.264138,13.0,438.0,139.0,20.43,15.0,51.889321,13.5,341.0,35.0,21.80,15.0,34.717991,14.0,154.0,8.0,23.04,15.0,15.013185,14.5,94.0,0.0,24.60,14.0,8.820343,15.0,222.0,18.0,26.82,13.0,21.957266
20757,10.5,180.0,704.0,36.47,14.0,58.739732,11.0,324.0,319.0,34.03,15.0,50.481835,11.5,350.0,271.0,34.01,15.0,50.575667,12.0,301.0,454.0,34.01,16.0,58.456947,12.5,366.0,157.0,36.27,15.0,44.758058,13.0,345.0,95.0,36.38,15.0,38.658952,13.5,103.0,0.0,38.83,...,504.0,49.34,0.0,8.540157,7.5,47.0,690.0,45.89,2.0,18.206088,8.0,58.0,791.0,37.35,4.0,28.059282,8.5,66.0,861.0,34.79,5.0,37.818616,9.0,74.0,901.0,28.37,6.0,46.733161,9.5,79.0,934.0,28.36,7.0,54.802944,10.0,87.0,948.0,26.47,8.0,61.558795
47627,5.5,0.0,0.0,89.06,7.0,0.000000,6.0,21.0,0.0,83.25,8.0,1.970632,6.5,43.0,399.0,77.71,9.0,9.196182,7.0,59.0,577.0,69.08,11.0,18.391960,7.5,72.0,683.0,60.43,13.0,28.056506,8.0,80.0,761.0,50.93,15.0,37.532963,8.5,87.0,811.0,44.74,...,0.0,74.39,8.0,0.000000,2.5,0.0,0.0,79.74,7.0,0.000000,3.0,0.0,0.0,80.88,7.0,0.000000,3.5,0.0,0.0,80.91,7.0,0.000000,4.0,0.0,0.0,82.34,7.0,0.000000,4.5,0.0,0.0,88.32,6.0,0.000000,5.0,0.0,0.0,89.49,6.0,0.000000
37951,15.5,169.0,330.0,46.07,4.0,28.528501,16.0,137.0,294.0,48.67,3.0,21.959679,16.5,97.0,264.0,52.33,1.0,15.109342,17.0,58.0,200.0,56.07,0.0,8.352461,17.5,20.0,96.0,56.08,0.0,2.440045,18.0,0.0,0.0,61.10,0.0,0.000000,18.5,0.0,0.0,61.10,...,1010.0,35.29,5.0,72.821951,12.5,83.0,1009.0,35.28,5.0,72.634266,13.0,80.0,1009.0,36.61,5.0,71.038940,13.5,244.0,522.0,36.60,5.0,54.522621,14.0,184.0,652.0,37.83,5.0,54.334935,14.5,72.0,967.0,37.83,5.0,57.150217,15.0,68.0,940.0,39.04,5.0,49.924327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11428,2.0,0.0,0.0,49.47,12.0,0.000000,2.5,0.0,0.0,49.46,12.0,0.000000,3.0,0.0,0.0,47.39,12.0,0.000000,3.5,0.0,0.0,47.39,11.0,0.000000,4.0,0.0,0.0,47.81,11.0,0.000000,4.5,0.0,0.0,47.82,11.0,0.000000,5.0,0.0,0.0,50.08,...,0.0,69.17,18.0,0.000000,23.0,0.0,0.0,69.72,18.0,0.000000,23.5,0.0,0.0,74.31,17.0,0.000000,0.0,0.0,0.0,74.06,17.0,0.000000,0.5,0.0,0.0,74.05,16.0,0.000000,1.0,0.0,0.0,73.60,16.0,0.000000,1.5,0.0,0.0,78.49,16.

In [44]:
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [90]:
from lightgbm import LGBMRegressor

# LGBM 모델을 구축하고 예측까지 하는 함수를 만든다. 
def LGBM(q, X_train, Y_train, X_test):
    
    # (a) 모델링
    model = LGBMRegressor(objective='quantile', alpha=q,
                         n_estimators=10000, bagging_fraction=0.7, learning_rate=0.027, subsample=0.7)          
    
        # bagging_fraction : 배깅을 하기위해서 데이터를 랜덤 샘플링하여 학습에 사용한다. 비율은 0 < fraction <= 1 이며 0이 되지 않게 해야한다.
        # learning_rate : 일반적으로 0.01 ~ 0.1 정도로 맞추고 다른 파라미터를 튜닝한다. 나중에 성능을 더 높일 때 learning rate를 더 줄인다.
        # n_estimators : 기본값은 100, 1000 정도로 해주는 게 좋고 많을 수록 과적합이 발생한 가능성이 높음.
        # https://greatjoy.tistory.com/72
        
    model.fit(X_train, Y_train, eval_metric = ['quantile'], verbose=500)


        # verbose : eval metric이 이 숫자만큼의 round가 지난 다음 자동으로 출력된다.
        # early_stopping_rounds : 이 숫자가 가기 전까지 validation score가 증가하지 않으면 round를 멈춘다. 

    # (b) 예측
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [91]:
# Target 예측

def train_data(X_train, Y_train, X_test):

    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q)
        pred , model = LGBM(q, X_train, Y_train, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred

In [92]:
# Target1
models_1, results_1 = train_data(X_train_1, Y_train_1, X_test)
results_1.sort_index()[:48]

0.1


KeyboardInterrupt: ignored

In [ ]:
# 시계열 분석의 백테스팅 - walk-forward validation for univariate data
# Cut Point를 정해서 그 전은 Train / 그 이후는 Test Set으로 정함.
# 하나의 Step을 정한 다음에, 예를 들어서 1 week forecast라고 하면 test의 데이터로 1 step 예측 후, 1 step 간 부분의 real data 까지 포함해서 그 다음 부분을 예측하는 방식.
def walk_forward_validation(data, n_test):
	predictions = list()
	# 데이터셋 분리
	train, test = train_test_split(data, n_test)
	# train 데이터셋의 seed 히스토리
	history = [x for x in train]
	# test set을 부분부분 train set에 포함시킴.
	for i in range(len(test)):
		# split test row into input and output columns
		testX, testy = test[i, :-1], test[i, -1]
		# fit model on history and make a prediction - One Step Forecast 수행함.
		yhat = xgboost_forecast(history, testX)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop - 여기서 actual observation을 history에 추가시킴.
		history.append(test[i])
		# summarize progress
		print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
	# estimate prediction error
	error = mean_absolute_error(test[:, -1], predictions)
	return error, test[:, 1], predictions

In [ ]:
#  XGBRegressor 사용하여 One-step 예측하는 함수를 구축함.

def xgboost_forecast(train, testX):
	# transform list into array
	train = asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1], train[:, -1]
	# fit model
	model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
	model.fit(trainX, trainy)
	# make a one-step prediction
	yhat = model.predict([testX])
	return yhat[0]


  ## 여기서 더 못나가겠다 무슨 말하는지 모르겠다 

In [ ]:
train_X, train_y, test_X, test_y = data_loader(raw_data)

In [ ]:
type(train_X)

torch.Tensor